In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Head_and_Neck_Cancer"
cohort = "GSE212250"

# Input paths
in_trait_dir = "../../input/GEO/Head_and_Neck_Cancer"
in_cohort_dir = "../../input/GEO/Head_and_Neck_Cancer/GSE212250"

# Output paths
out_data_file = "../../output/preprocess/Head_and_Neck_Cancer/GSE212250.csv"
out_gene_data_file = "../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE212250.csv"
out_clinical_data_file = "../../output/preprocess/Head_and_Neck_Cancer/clinical_data/GSE212250.csv"
json_path = "../../output/preprocess/Head_and_Neck_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Deciphering the function of intrinsic and genomics-driven epigenetic heterogeneity in head and neck cancer progression with single-nucleus CUT&RUN"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell line: HN120Pri', 'cell line: HN120Met', 'cell line: HN120PCR', 'cell line: HN137Pri', 'cell line: HN137Met', 'cell line: HN137PCR'], 1: ['cell type: Primary patient derived oral cancer cell line'], 2: ['disease state: Primary tumor', 'disease state: Metastatic tumor', 'disease state: Primary Cisplatin Resistant tumor'], 3: ['antibody: H3K4me3 (Abcam, ab213224)', 'antibody: H3K27ac (Merck, MABE647)']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze the dataset to determine gene expression, trait, age, and gender availability

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains CUT&RUN data for histone modifications (H3K4me3, H3K27ac)
# from head and neck cancer cell lines, which is epigenetic data rather than gene expression data
is_gene_available = False  # Epigenetic data, not gene expression data

# 2. Clinical Feature Availability and Conversion
# 2.1 Trait Availability
# Looking at the sample characteristics, I can identify:
# Key 2 contains disease state information relevant to Head and Neck Cancer
trait_row = 2  # Maps to "disease state: Primary tumor", "disease state: Metastatic tumor", etc.

# Age information is not available in the sample characteristics
age_row = None

# Gender information is not available in the sample characteristics 
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert disease state values to binary format:
    1 = Metastatic tumor (more severe)
    0 = Primary tumor (less severe)
    None = Other or unknown
    """
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "Metastatic" in value:
        return 1  # Metastatic tumor
    elif "Primary" in value:
        return 0  # Primary tumor
    else:
        return None  # Other or unknown disease state

def convert_age(value):
    """Placeholder function since age data is not available"""
    return None

def convert_gender(value):
    """Placeholder function since gender data is not available"""
    return None

# 3. Save Metadata - Initial Filtering
# Determine trait data availability
is_trait_available = trait_row is not None

# Save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step if trait_row is None
if trait_row is not None:
    # We need clinical_data from a previous step
    try:
        # Extract and process clinical features
        clinical_df = geo_select_clinical_features(
            clinical_data,  # Assumed to be defined in previous step
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the clinical data
        preview = preview_df(clinical_df)
        print("Clinical Data Preview:")
        print(preview)
        
        # Ensure the output directory exists
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical data to CSV
        clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except NameError:
        print("Warning: clinical_data not defined in previous steps. Skipping clinical feature extraction.")


Clinical Data Preview:
{'GSM6515847': [0.0], 'GSM6515848': [0.0], 'GSM6515849': [0.0], 'GSM6515850': [0.0], 'GSM6515851': [0.0], 'GSM6515852': [0.0], 'GSM6515853': [0.0], 'GSM6515854': [0.0], 'GSM6515855': [0.0], 'GSM6515856': [0.0], 'GSM6515857': [0.0], 'GSM6515858': [0.0], 'GSM6515859': [0.0], 'GSM6515860': [0.0], 'GSM6515861': [0.0], 'GSM6515862': [0.0], 'GSM6515863': [0.0], 'GSM6515864': [0.0], 'GSM6515865': [0.0], 'GSM6515866': [0.0], 'GSM6515867': [0.0], 'GSM6515868': [0.0], 'GSM6515869': [0.0], 'GSM6515870': [0.0], 'GSM6515871': [0.0], 'GSM6515872': [0.0], 'GSM6515873': [0.0], 'GSM6515874': [0.0], 'GSM6515875': [0.0], 'GSM6515876': [0.0], 'GSM6515877': [0.0], 'GSM6515878': [0.0], 'GSM6515879': [0.0], 'GSM6515880': [0.0], 'GSM6515881': [0.0], 'GSM6515882': [0.0], 'GSM6515883': [0.0], 'GSM6515884': [0.0], 'GSM6515885': [0.0], 'GSM6515886': [0.0], 'GSM6515887': [0.0], 'GSM6515888': [0.0], 'GSM6515889': [0.0], 'GSM6515890': [0.0], 'GSM6515891': [0.0], 'GSM6515892': [0.0], 'GSM651589

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Head_and_Neck_Cancer/GSE212250/GSE212250-GPL15520_series_matrix.txt.gz


Gene data shape: (0, 3955)
First 20 gene/probe identifiers:
Index([], dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the analysis of the file, we've found that GSE212250 is a SuperSeries
# which doesn't contain direct gene expression data itself but refers to multiple SubSeries.
# The file appears to be a microRNA dataset (GPL15520 - Agilent-045997 Arraystar miRNA platform)
# and doesn't contain gene identifiers in the expected format.

# We cannot determine the need for gene mapping when no gene data is available
# The gene data shape (0, 3955) indicates no gene rows were found in the dataset

# Since we don't have actual gene identifiers to evaluate, we should indicate that 
# gene data is not available rather than determining if mapping is needed

requires_gene_mapping = False  # This is not relevant as gene data is not available

# Record that gene data is unavailable for this dataset
is_gene_available = False
is_trait_available = False  # We haven't confirmed trait data either

# Save this information about the dataset
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

False